# Result Arrangement

In [1]:
import pandas as pd
from glob import glob
import os
import pickle
import sys
import re
import dask.dataframe as dd
from collections import Counter
from sklearn.metrics import accuracy_score, classification_report, roc_curve
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
def CounterOutcome(row, categoryClass):
    from collections import Counter
    id_ = row.name
    target = row[['LogisticRegression',
       'NeuralNetwork_1', 'NeuralNetwork_2', 'RandomForestClassifier',
       'SVC', 'XGBoost']]
    counter = dict(Counter(target))
    notInCounter = list(filter(lambda x: x not in counter.keys(), categoryClass.classes_))
    others = dict(zip(notInCounter, [0] * len(notInCounter)))
    counter.update(others)
    counter['id'] = id_
    return counter

In [3]:
plt.rcParams["font.family"]

['NanumGothicOTF']

In [4]:
import Basic_Module as bm

In [5]:
def Get_Daum_Category(row, odf):
    from bson import ObjectId
    import requests
    from bs4 import BeautifulSoup
    import numpy as np
    link = odf.loc[row.name,'link']
    res = requests.get(link)
    soup = BeautifulSoup(res.content, 'html.parser')
    try:
        cat = soup.find('meta', property = "dg:uoc:svc").attrs['content']
        if cat == 'news':
            real = soup.find('ul', class_ = 'gnb_comm').attrs['data-category']
        elif cat == 'entertain':
            real = '연예'
        else:
            real = '스포츠'
    except:
        real = np.NAN
    return real

## 다음 뉴스의 실제 카테고리 얻기

In [6]:
if not os.path.isfile('./data/pre_data/stastics/real_category_daum_news.pickled'):
    import Database_Handler as dh
    from bson import ObjectId
    import requests
    from bs4 import BeautifulSoup
    mongodb = dh.ToMongoDB(*dh.GCP_MongoDB_Information())
    dbname = 'hy_db'
    useDB = dh.Use_Database(mongodb, dbname)
    newsCollection = dh.Use_Collection(useDB, 'newsDaum')
    cursor = newsCollection.find({})
    li = list()
    for idx, cur in enumerate(cursor):
        if idx % 1000 == 0:
            print (idx)
        li.append((cur['_id'],cur['link']))
    lidf = pd.DataFrame.from_records(li)
    lidf[0] = lidf[0].apply(lambda x: x._ObjectId__id.hex())
    lidf.rename({0 : 'id', 1 : 'link'}, inplace=True, axis = 1)
    lidf.set_index('id', inplace = True)
    real_Category = []
    for enu, idx in enumerate(outdf.index):
        if enu % 1000 == 0:
            print (enu)
        real = Get_Daum_Category(outdf.loc[idx], lidf)
        real_Category.append((idx, real))
    pickle.dump(real_Category,open('./data/pre_data/stastics/real_category_daum_news.pickled','wb'))
    lidf.to_csv('./data/pre_data/stastics/id_link_daum_news.csv', index=True, header = True, encoding='utf-8')
else:
    real_Category = pickle.load(open('./data/pre_data/stastics/real_category_daum_news.pickled','rb'))
    lidf = pd.read_csv('./data/pre_data/stastics/id_link_daum_news.csv', index_col=0, encoding='utf-8')

In [7]:
catDict = {
    'foreign' : '세계',
    'culture' : '생활/문화',
    'society' : '사회',
    'economic' : '경제',
    'digital' : 'IT/과학',
    'politics' : '정치',
    'nation' : '사회',
    'stock' : '경제',
    'breakingnews' : '스포츠',
    '연예' : '연예',
    '스포츠' : '스포츠'
}

* foreign : 세계
* culture : 생활/문화
* society : 사회
* economic : 경제
* digital : IT/과학
* politics : 정치
* nation : ?
* 연예 : 연예
* 스포츠 : 스포츠
* stock : 경제
* breakingnews :
> 1건  
> * 내용은 컬링 경기에서 사용되는 돌에 대한 기사 스포츠? 세계? -> 스포츠로 둔다. 
* nan 
> 언론사의 요청으로 기사가 삭제된 기사 3건  
> 제휴종료된 언론사의 기사 1건

In [8]:
if not os.path.isfile('./data/pre_data/stastics/real_category_daum_news.csv'):
    rcdf = pd.DataFrame.from_records(real_Category)
    rcdf.rename({0 : 'id', 1 : 'real_category'}, inplace=True, axis = 1)
    rcdf.set_index('id', inplace = True)
    rcdf = rcdf.merge(lidf, right_index=True, left_index=True)
    rcdf['real_category_from_naver'] = rcdf.real_category.map(catDict)
    rcdf.to_csv('./data/pre_data/stastics/real_category_daum_news.csv', index=True, header = True, encoding='utf-8')
    print (rcdf.shape)
else:
    rcdf = pd.read_csv('./data/pre_data/stastics/real_category_daum_news.csv')
    rcdf.set_index('id', inplace = True)
    print (rcdf.shape)

(9372, 3)


In [9]:
rcdf.real_category.fillna('null', inplace=True)

In [10]:
rcdf[rcdf.real_category.isna()]

,real_category,link,real_category_from_naver
id,,,


In [11]:
print (rcdf.real_category_from_naver.unique())

['세계' '생활/문화' '사회' '경제' 'IT/과학' '정치' '연예' '스포츠' nan]


In [12]:
rcdf.real_category_from_naver.fillna('null', inplace=True)

In [13]:
rcdf[rcdf.real_category_from_naver.isna()]

,real_category,link,real_category_from_naver
id,,,


In [14]:
print (rcdf.real_category.unique())

['foreign' 'culture' 'society' 'economic' 'digital' 'politics' 'nation'
 '연예' '스포츠' 'stock' 'null' 'breakingnews']


In [15]:
rcdf.head()

,real_category,link,real_category_from_naver
id,,,
5a2a61bf588c13481c229d1e,foreign,http://v.media.daum.net/v/20171207100332307,세계
5a2a61bf588c13481c229d1f,culture,http://v.media.daum.net/v/20171207135520573,생활/문화
5a2a61bf588c13481c229d20,society,http://v.media.daum.net/v/20171207093806281,사회
5a2a61bf588c13481c229d21,economic,http://v.media.daum.net/v/20171207145326949,경제
5a2a61bf588c13481c229d22,society,http://v.media.daum.net/v/20171207171708285,사회


## Category 

In [16]:
if os.path.isfile('./data/pre_data/news_tagged_data/pre_data_category_label_encoder_by_ct_for_doc2vec_news_classification.pickled'):
    le = pickle.load(open('./data/pre_data/news_tagged_data/pre_data_category_label_encoder_by_ct_for_doc2vec_news_classification.pickled','rb'))
else:
    le = LabelEncoder()
    le.fit(naverData['category'])
    pickle.dump(le, open('./data/pre_data/news_tagged_data/pre_data_category_label_encoder_by_ct_for_doc2vec_news_classification.pickled','wb'))
print (le.classes_)

['IT/과학' '경제' '사회' '생활/문화' '세계' '스포츠' '연예' '정치']


## Path

In [17]:
if sys.platform =='darwin':
    loadModelPath = '/Volumes/disk1/news_model/'
elif sys.platform =='win32':
    loadModelPath = 'd:/news_model/'
daumNewsPath = './data/pre_data/news_daum_news/'
classifierPath = './data/pre_data/news_classifier/'
outpath = './outcome_news_classification/'

In [18]:
outcomeFileList = glob(outpath+'outcome_news_classification'+'*')
print (len(outcomeFileList))

18


## 모델 및 Classifier 비교
> ### 1. model별로 6개의 classifier중에서 가장 많이 분류된 category를 통해 실제 category와 비교한 Accuracy score  
> ### 2. model별 각 classifier가 분류한 category와 실제 category를 비교한 Accuracy score

In [19]:
outdict = dict()
for idx, outcomeFile in enumerate(sorted(outcomeFileList)):
    fileName = os.path.split(outcomeFile)[1]
    df = pd.read_csv(outcomeFile, index_col = None, encoding='utf-8')
    df.set_index(keys='id', inplace=True)
    out = df.apply(lambda x: CounterOutcome(x, le), axis = 1)
    out = pd.DataFrame.from_records(out, columns=['id','정치','경제','사회','생활/문화','세계','IT/과학','연예','스포츠'])
    out.set_index(keys='id', inplace = True)
    modelIs, tagger = bm.ExtractModelType(outcomeFile)
    matchCount = (out.idxmax(axis = 1) == rcdf.real_category_from_naver).sum()
    acc = accuracy_score(rcdf.real_category_from_naver, out.idxmax(axis = 1))
    print (
    '''* {}. \n* Model : {}\n* tagger : {}\n* Match count : {} / {}\n* Accuracy score : {:.3f}'''.format(idx, modelIs, tagger, matchCount, out.shape[0], acc))
    byCls = df.loc[:,['LogisticRegression',
       'NeuralNetwork_1', 'NeuralNetwork_2', 'RandomForestClassifier',
       'SVC', 'XGBoost']].apply(lambda x: accuracy_score(rcdf.real_category_from_naver, x))
    byCls = byCls.apply(lambda x: '{:.3f}'.format(x))
    print (byCls)
    print ()
    mx = '{}_{}'.format(modelIs, tagger)
    outdict[mx] = out

* 0. 
* Model : Doc2Vec_dbow
* tagger : twitter
* Match count : 8025 / 9372
* Accuracy score : 0.856
LogisticRegression        0.883
NeuralNetwork_1           0.803
NeuralNetwork_2           0.783
RandomForestClassifier    0.800
SVC                       0.886
XGBoost                   0.767
dtype: object

* 1. 
* Model : Doc2Vec_dbow
* tagger : mecab
* Match count : 8017 / 9372
* Accuracy score : 0.855
LogisticRegression        0.885
NeuralNetwork_1           0.789
NeuralNetwork_2           0.786
RandomForestClassifier    0.809
SVC                       0.890
XGBoost                   0.772
dtype: object

* 2. 
* Model : Doc2Vec_dm-c
* tagger : twitter
* Match count : 6304 / 9372
* Accuracy score : 0.673
LogisticRegression        0.704
NeuralNetwork_1           0.631
NeuralNetwork_2           0.560
RandomForestClassifier    0.639
SVC                       0.667
XGBoost                   0.584
dtype: object

* 3. 
* Model : Doc2Vec_dm-c
* tagger : mecab
* Match count : 6358 / 9372
* Ac

##  각 뉴스에 대하여 총 108개의 classifier가 분류한 Category 중에서 가장 많이 분류된 category가 실제 category와 일치하는 비율

In [20]:
print (len(outdict.keys()))

18


In [21]:
outdf = pd.DataFrame()
for idx,key in enumerate(outdict):
    data = outdict[key]
    if outdf.shape == (0, 0):
        outdf = data.copy()
    else:
        outdf = outdf.add(data)
allAcc = accuracy_score(rcdf.real_category_from_naver, outdf.idxmax(axis = 1))
matchCount = (outdf.idxmax(axis = 1) == rcdf.real_category_from_naver).sum()
print ('Accuracy score : {:.4f}'.format(allAcc))
print ('Match count : {} / {}'.format(matchCount, outdf.shape[0]))
print (classification_report(rcdf.real_category_from_naver, outdf.idxmax(axis = 1)))
outdf.head()

Accuracy score : 0.8556
Match count : 8019 / 9372
             precision    recall  f1-score   support

      IT/과학       0.27      0.75      0.40        65
       null       0.00      0.00      0.00         4
         경제       0.58      0.71      0.64       331
         사회       0.81      0.73      0.77      1456
      생활/문화       0.39      0.77      0.51       218
         세계       0.52      0.84      0.64       330
        스포츠       1.00      0.92      0.95      3141
         연예       0.99      0.88      0.93      3079
         정치       0.72      0.86      0.79       748

avg / total       0.89      0.86      0.87      9372



/Users/hyunyoun/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


,정치,경제,사회,생활/문화,세계,IT/과학,연예,스포츠
id,,,,,,,,
5a2a61bf588c13481c229d1e,0,1,25,21,60,1,0,0
5a2a61bf588c13481c229d1f,0,0,0,108,0,0,0,0
5a2a61bf588c13481c229d20,1,0,106,0,1,0,0,0
5a2a61bf588c13481c229d21,1,89,14,0,0,1,3,0
5a2a61bf588c13481c229d22,0,0,108,0,0,0,0,0


## classifier의 수

In [22]:
clsIs = outdf.sum(axis = 1).values
clsCount = np.unique(clsIs)[0]
print ('classifier의 수 : {} 개'.format(clsCount))

classifier의 수 : 108 개


In [23]:
lstOutDf = df.loc[:,['title','extracted_keywords']]
lstOutDf = lstOutDf.merge(outdf, right_index=True, left_index=True)
lstOutDf.head()

,title,extracted_keywords,정치,경제,사회,생활/문화,세계,IT/과학,연예,스포츠
id,,,,,,,,,,
5a2a61bf588c13481c229d1e,"""밤이 무섭다""..비아그라 공장 연기에 남성들 부작용 호소","{'공장', '건강', '남성들', '지역', '부작용', '연기', '세보 효과'}",0,1,25,21,60,1,0,0
5a2a61bf588c13481c229d1f,식후 커피·늦은 양치질..점심식사 후 하면 안 좋은 습관 3가지,"{'치아', '건강', '입냄새', '커피', '점심 식사', '낮잠', '디스크'...",0,0,0,108,0,0,0,0
5a2a61bf588c13481c229d20,"'십년지기 생매장' 진짜 이유는..""'청부 통정' 알려질까 봐""","{'경찰', '성관계', '지인', '앙심', '진술', '철원', '아들', '주...",1,0,106,0,1,0,0,0
5a2a61bf588c13481c229d21,"신영자, 억 소리나는 갑질","{'신영자 이사장', '혐의', '롯데', '검찰', '유통업체', '징역', '매...",1,89,14,0,0,1,3,0
5a2a61bf588c13481c229d22,"""배신하지마"" 20대女 살인 피의자 유치장서 공범 남친에 쪽지","{'경찰', '혐의', '폭행', '쪽지', '유치장', '남자친구', '과자', ...",0,0,108,0,0,0,0,0


## 여러 classifier로 다음 뉴스를 네이버 category로 분류해보았을때, 모든 classifier가 하나의 cateogy로 분류한 비율

In [24]:
oneCategoryOutput = lstOutDf[outdf.eq(clsCount).sum(axis = 1) == 1]
oneCategoryOutput.head()

,title,extracted_keywords,정치,경제,사회,생활/문화,세계,IT/과학,연예,스포츠
id,,,,,,,,,,
5a2a61bf588c13481c229d1f,식후 커피·늦은 양치질..점심식사 후 하면 안 좋은 습관 3가지,"{'치아', '건강', '입냄새', '커피', '점심 식사', '낮잠', '디스크'...",0,0,0,108,0,0,0,0
5a2a61bf588c13481c229d22,"""배신하지마"" 20대女 살인 피의자 유치장서 공범 남친에 쪽지","{'경찰', '혐의', '폭행', '쪽지', '유치장', '남자친구', '과자', ...",0,0,108,0,0,0,0,0
5a2a61bf588c13481c229d40,"""사장님과 식사할 땐 턱받이까지 매드려야 해요""","{'장기자랑', '직원들', '사례', '119', '제보', '병원', '직장 갑질'}",0,0,108,0,0,0,0,0
5a2a61bf588c13481c229d58,"'마이웨이' 박해미 ""아픔까지 안아 준 9세 연하 남편..결혼식 못 올렸다""","{'마이웨이', '인생', '박해미', '배우', '열정', '남편 황민', '사랑'}",0,0,0,0,0,0,108,0
5a2a61bf588c13481c229d77,"'감빵생활' 박해수, 정수정 버렸다..이규형x정해인 반전 [종합]","{'준호', '박해수', '감빵 생활', '제혁', '지호', '야구', '슬기'}",0,0,0,0,0,0,108,0


In [25]:
oneProp = oneCategoryOutput.shape[0]/lstOutDf.shape[0]
print ('Match count : {} / {}'.format(oneCategoryOutput.shape[0], lstOutDf.shape[0]))
print ('비율 : {0:.4f}, 약 {1:.1f}%'.format(oneProp, oneProp * 100))
print ()
oneAcc = accuracy_score(rcdf.loc[oneCategoryOutput.index.values, 'real_category_from_naver'], outdf.loc[oneCategoryOutput.index.values].idxmax(axis = 1))
print ('Accuracy score : {:.4f}'.format(oneAcc))
print (classification_report(rcdf.loc[oneCategoryOutput.index.values, 'real_category_from_naver'], outdf.loc[oneCategoryOutput.index.values].idxmax(axis = 1)))

Match count : 1038 / 9372
비율 : 0.1108, 약 11.1%

Accuracy score : 0.9855
             precision    recall  f1-score   support

      IT/과학       0.76      0.93      0.84        14
         경제       0.93      0.90      0.92        30
         사회       1.00      0.93      0.96        96
      생활/문화       0.75      1.00      0.86         6
         세계       1.00      0.93      0.97        15
        스포츠       1.00      1.00      1.00       663
         연예       0.99      0.99      0.99       154
         정치       0.91      0.98      0.94        60

avg / total       0.99      0.99      0.99      1038



## 여러 classifier로 다음 뉴스를 네이버 category로 분류해보았을때, 특정 category가 75%이상 나온 비율

In [26]:
x = outdf.divide(clsCount).multiply(100) >= 75
gt75 = lstOutDf[x.sum(axis = 1) == 1]
gt75.head()

,title,extracted_keywords,정치,경제,사회,생활/문화,세계,IT/과학,연예,스포츠
id,,,,,,,,,,
5a2a61bf588c13481c229d1f,식후 커피·늦은 양치질..점심식사 후 하면 안 좋은 습관 3가지,"{'치아', '건강', '입냄새', '커피', '점심 식사', '낮잠', '디스크'...",0,0,0,108,0,0,0,0
5a2a61bf588c13481c229d20,"'십년지기 생매장' 진짜 이유는..""'청부 통정' 알려질까 봐""","{'경찰', '성관계', '지인', '앙심', '진술', '철원', '아들', '주...",1,0,106,0,1,0,0,0
5a2a61bf588c13481c229d21,"신영자, 억 소리나는 갑질","{'신영자 이사장', '혐의', '롯데', '검찰', '유통업체', '징역', '매...",1,89,14,0,0,1,3,0
5a2a61bf588c13481c229d22,"""배신하지마"" 20대女 살인 피의자 유치장서 공범 남친에 쪽지","{'경찰', '혐의', '폭행', '쪽지', '유치장', '남자친구', '과자', ...",0,0,108,0,0,0,0,0
5a2a61bf588c13481c229d23,"화성서 발견된 포탄 모양 둥근 물체, 정체는?","{'공개', '모양 물체', '사진', 'NASA', '학자', '암석', '화성'}",0,0,1,2,3,102,0,0


In [27]:
prop2 = gt75.shape[0]/lstOutDf.shape[0]
print ('Match count : {} / {}'.format(gt75.shape[0], lstOutDf.shape[0]))
print ('비율 : {0:.4f}, 약 {1:.1f}%'.format(prop2, prop2 * 100))
print ()
accis = accuracy_score(rcdf.loc[gt75.index.values, 'real_category_from_naver'], outdf.loc[gt75.index.values].idxmax(axis = 1))
print ('Accuracy score : {:.4f}'.format(accis))
print (classification_report(rcdf.loc[gt75.index.values, 'real_category_from_naver'], outdf.loc[gt75.index.values].idxmax(axis = 1)))

Match count : 7422 / 9372
비율 : 0.7919, 약 79.2%

Accuracy score : 0.9305
             precision    recall  f1-score   support

      IT/과학       0.49      0.82      0.61        56
       null       0.00      0.00      0.00         3
         경제       0.74      0.77      0.76       266
         사회       0.92      0.78      0.85      1098
      생활/문화       0.63      0.91      0.75       151
         세계       0.76      0.91      0.83       244
        스포츠       1.00      0.98      0.99      2649
         연예       0.99      0.98      0.99      2338
         정치       0.79      0.91      0.85       617

avg / total       0.94      0.93      0.93      7422



/Users/hyunyoun/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


## 여러 classifier로 다음 뉴스를 네이버 category로 분류해보았을때, 특정 category가 50%보다 높게 나온 비율

In [28]:
x = outdf.divide(clsCount).multiply(100) > 50
gt50 = lstOutDf[x.sum(axis = 1) == 1]
gt50.head()

,title,extracted_keywords,정치,경제,사회,생활/문화,세계,IT/과학,연예,스포츠
id,,,,,,,,,,
5a2a61bf588c13481c229d1e,"""밤이 무섭다""..비아그라 공장 연기에 남성들 부작용 호소","{'공장', '건강', '남성들', '지역', '부작용', '연기', '세보 효과'}",0,1,25,21,60,1,0,0
5a2a61bf588c13481c229d1f,식후 커피·늦은 양치질..점심식사 후 하면 안 좋은 습관 3가지,"{'치아', '건강', '입냄새', '커피', '점심 식사', '낮잠', '디스크'...",0,0,0,108,0,0,0,0
5a2a61bf588c13481c229d20,"'십년지기 생매장' 진짜 이유는..""'청부 통정' 알려질까 봐""","{'경찰', '성관계', '지인', '앙심', '진술', '철원', '아들', '주...",1,0,106,0,1,0,0,0
5a2a61bf588c13481c229d21,"신영자, 억 소리나는 갑질","{'신영자 이사장', '혐의', '롯데', '검찰', '유통업체', '징역', '매...",1,89,14,0,0,1,3,0
5a2a61bf588c13481c229d22,"""배신하지마"" 20대女 살인 피의자 유치장서 공범 남친에 쪽지","{'경찰', '혐의', '폭행', '쪽지', '유치장', '남자친구', '과자', ...",0,0,108,0,0,0,0,0


In [29]:
prop3 = gt50.shape[0]/lstOutDf.shape[0]
print ('Match count : {} / {}'.format(gt50.shape[0], lstOutDf.shape[0]))
print ('비율 : {0:.4f}, 약 {1:.1f}%'.format(prop3, prop3 * 100))
print ()
accis = accuracy_score(rcdf.loc[gt50.index.values, 'real_category_from_naver'], outdf.loc[gt50.index.values].idxmax(axis = 1))
print ('Accuracy score : {:.4f}'.format(accis))
print (classification_report(rcdf.loc[gt50.index.values, 'real_category_from_naver'], outdf.loc[gt50.index.values].idxmax(axis = 1)))

Match count : 8781 / 9372
비율 : 0.9369, 약 93.7%

Accuracy score : 0.8830
             precision    recall  f1-score   support

      IT/과학       0.36      0.77      0.49        64
       null       0.00      0.00      0.00         4
         경제       0.63      0.72      0.67       320
         사회       0.85      0.74      0.79      1398
      생활/문화       0.46      0.81      0.59       198
         세계       0.59      0.87      0.70       313
        스포츠       1.00      0.94      0.97      2954
         연예       0.99      0.92      0.96      2811
         정치       0.74      0.88      0.80       719

avg / total       0.91      0.88      0.89      8781



/Users/hyunyoun/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


## 여러 classifier로 다음 뉴스를 네이버 category로 분류해보았을때, 특정 category가 50%인 뉴스의 비율

In [30]:
x = outdf.divide(clsCount).multiply(100) == 50

eq50 = lstOutDf[x.sum(axis = 1) == 1]
eq50.head()

,title,extracted_keywords,정치,경제,사회,생활/문화,세계,IT/과학,연예,스포츠
id,,,,,,,,,,
5a2aec8b588c132078bd207d,[뉴스+] 주말에 '집돌이·집순이' 되는 직장인들,"{'주말', '번아웃 증후군', '직장인', '스트레스', '증상', '교수', '...",0,2,54,48,3,1,0,0
5a2af415588c1376f0d67b16,'박근혜 5촌살인 의혹보도' 주진우·김어준 내일 대법원 선고,"{'사건', '연루', '혐의', '선고', '기자', '보도', '대통령', '조...",54,1,45,0,3,2,3,0
5a2b0aa2588c136a080754ec,"강남소방서 측 ""태연 가슴통증 호소..응급처치 특혜NO""","{'교통사고', '강남 소방서', '가슴 통증', '태연', '특혜'}",5,3,40,3,0,3,54,0
5a2b0eac588c13738825a5c2,"황우슬혜 ""꿀피부 비결은 아기 화장품, 비싼 것 필요 없어""","{'질문', '도전', '아기 화장품', '황우슬혜', '피부 관리', '역할'}",0,5,5,54,8,9,27,0
5a2c3b55588c135f3cb21ca8,"'이방인' 추신수 아내 하원미 ""삼남매 한방서 자는 것, 첫째 분리불안 때문""","{'김예은', '사진 이방인 캡처', '하원미', '경기', '삼남매', '이방인 ...",0,0,7,4,54,0,42,1


In [31]:
prop3 = eq50.shape[0]/lstOutDf.shape[0]
print ('Match count : {} / {}'.format(eq50.shape[0], lstOutDf.shape[0]))
print ('비율 : {0:.4f}, 약 {1:.1f}%'.format(prop3, prop3 * 100))
print ()
accis = accuracy_score(rcdf.loc[eq50.index.values, 'real_category_from_naver'], outdf.loc[eq50.index.values].idxmax(axis = 1))
print ('Accuracy score : {:.4f}'.format(accis))
print (classification_report(rcdf.loc[eq50.index.values, 'real_category_from_naver'], outdf.loc[eq50.index.values].idxmax(axis = 1)))

Match count : 50 / 9372
비율 : 0.0053, 약 0.5%

Accuracy score : 0.5000
             precision    recall  f1-score   support

      IT/과학       0.00      0.00      0.00         0
         경제       0.00      0.00      0.00         1
         사회       0.33      0.67      0.44         6
      생활/문화       0.00      0.00      0.00         1
         세계       0.12      0.50      0.20         2
        스포츠       1.00      0.67      0.80         9
         연예       1.00      0.48      0.65        25
         정치       0.67      0.33      0.44         6

avg / total       0.81      0.50      0.58        50



/Users/hyunyoun/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


## 여러 classifier로 다음 뉴스를 네이버 category로 분류해보았을때, 특정 category가 50%이상인 것이 하나도 없는 뉴스의 비율

In [32]:
x = outdf.divide(clsCount).multiply(100) >= 50

lt50 = lstOutDf[x.sum(axis = 1) == 0]
lt50.head()

,title,extracted_keywords,정치,경제,사회,생활/문화,세계,IT/과학,연예,스포츠
id,,,,,,,,,,
5a2a61bf588c13481c229d4f,"문 대통령, '에너지제로 주택 신혼부부 입주세대 방문'","{'주택', '신혼부부', '서울 노원구', '대통령', '오픈 하우스', '에너지...",46,36,0,8,3,2,13,0
5a2a61bf588c13481c229d50,"[인터뷰]MBC 신동진 아나운서 ""6년만의 방송, 울컥하면 안되는데..""","{'부당 전보', '파업', '방송', '아나운서', '부서', '사장', '다시'}",2,4,32,24,1,20,25,0
5a2a61bf588c13481c229d5a,"[단독] ""윤계상, 합의 제안했다"" 탈세 주장 시위자, '무고죄 맞고소' (전문)","{'신고', '침대 업체', '허위', '소송', '주장', '윤계상 탈세'}",1,20,12,3,1,19,52,0
5a2a61bf588c13481c229d62,"이창명 ""그날 사건, 술 아닌 하늘의 경고""(인터뷰②)","{'편집', '스타', '이창명', '녹화', '만원', '전진학', '출발 드림팀...",5,9,33,17,3,4,37,0
5a2a61bf588c13481c229d6f,"'라디오스타' 손동운 ""회사 지분 51%, 매출 1위는 윤두준""","{'하이라이트 손동운', '51 매출', '멤버', '회사'}",1,17,3,0,6,36,45,0


In [33]:
prop3 = lt50.shape[0]/lstOutDf.shape[0]
print ('Match count : {} / {}'.format(lt50.shape[0], lstOutDf.shape[0]))
print ('비율 : {0:.4f}, 약 {1:.1f}%'.format(prop3, prop3 * 100))
print ()
accis = accuracy_score(rcdf.loc[lt50.index.values, 'real_category_from_naver'], outdf.loc[lt50.index.values].idxmax(axis = 1))
print ('Accuracy score : {:.4f}'.format(accis))
print (classification_report(rcdf.loc[lt50.index.values, 'real_category_from_naver'], outdf.loc[lt50.index.values].idxmax(axis = 1)))

Match count : 541 / 9372
비율 : 0.0577, 약 5.8%

Accuracy score : 0.4436
             precision    recall  f1-score   support

      IT/과학       0.00      0.00      0.00         1
         경제       0.13      0.50      0.21        10
         사회       0.26      0.42      0.32        52
      생활/문화       0.09      0.37      0.14        19
         세계       0.08      0.33      0.13        15
        스포츠       0.99      0.44      0.61       178
         연예       0.93      0.46      0.61       243
         정치       0.32      0.48      0.39        23

avg / total       0.79      0.44      0.54       541



## 결론

* Word2Vec, Doc2Vec, FastText로부터 만들어진 모델을 통해 단어 또는 문서를 vectorization을 하고 이를 통해 category를 분류해보았다. 
* 총 108개의 classifier를 통해 단일 category가 50%보다 높게 나올때 accuracy가 0.88정도가 나오는 것을 확인하였고, 75%이상일때는 0.93, 100%일 때는 0.98이 나오는 것을 확인할 수 있었다. 
* 카테고리중에서 50%를 넘는 것이 없는 경우에는 정확도가 0.44로 확연히 감소하는 것을 확인할 수 있었다. 

## 번외

In [34]:
lstOutDf2 = lstOutDf.merge(rcdf, left_index=True, right_index=True)
lstOutDf2['Decision'] = outdf.idxmax(axis = 1)
lstOutDf2.head()

,title,extracted_keywords,정치,경제,사회,생활/문화,세계,IT/과학,연예,스포츠,real_category,link,real_category_from_naver,Decision
id,,,,,,,,,,,,,,
5a2a61bf588c13481c229d1e,"""밤이 무섭다""..비아그라 공장 연기에 남성들 부작용 호소","{'공장', '건강', '남성들', '지역', '부작용', '연기', '세보 효과'}",0,1,25,21,60,1,0,0,foreign,http://v.media.daum.net/v/20171207100332307,세계,세계
5a2a61bf588c13481c229d1f,식후 커피·늦은 양치질..점심식사 후 하면 안 좋은 습관 3가지,"{'치아', '건강', '입냄새', '커피', '점심 식사', '낮잠', '디스크'...",0,0,0,108,0,0,0,0,culture,http://v.media.daum.net/v/20171207135520573,생활/문화,생활/문화
5a2a61bf588c13481c229d20,"'십년지기 생매장' 진짜 이유는..""'청부 통정' 알려질까 봐""","{'경찰', '성관계', '지인', '앙심', '진술', '철원', '아들', '주...",1,0,106,0,1,0,0,0,society,http://v.media.daum.net/v/20171207093806281,사회,사회
5a2a61bf588c13481c229d21,"신영자, 억 소리나는 갑질","{'신영자 이사장', '혐의', '롯데', '검찰', '유통업체', '징역', '매...",1,89,14,0,0,1,3,0,economic,http://v.media.daum.net/v/20171207145326949,경제,경제
5a2a61bf588c13481c229d22,"""배신하지마"" 20대女 살인 피의자 유치장서 공범 남친에 쪽지","{'경찰', '혐의', '폭행', '쪽지', '유치장', '남자친구', '과자', ...",0,0,108,0,0,0,0,0,society,http://v.media.daum.net/v/20171207171708285,사회,사회


## 다음 카테고리 중 아래 4개중 하나로 나온 뉴스 목록
* nation :
* stock :
* breakingnews :
> 1건  
> * 내용은 컬링 경기에서 사용되는 돌에 대한 기사 스포츠? 세계? -> 스포츠로 둔다. 
* null
> 언론사의 요청으로 기사가 삭제된 기사 3건  
> 제휴종료된 언론사의 기사 1건

## 실제 카테고리 : Null
> 4개

In [35]:
lstOutDf2[lstOutDf2.real_category == 'null']

,title,extracted_keywords,정치,경제,사회,생활/문화,세계,IT/과학,연예,스포츠,real_category,link,real_category_from_naver,Decision
id,,,,,,,,,,,,,,
5a316f43588c131bc8f2bdda,"[진단IS] 4년 째 망가짐만 고집한 고아라, 이제 영리함 필요할 때","{'망가짐', '연기력', '장면', '블랙', '응답 1994', '죽음', '고...",0,1,0,0,0,0,107,0,null,http://v.media.daum.net/v/20171213080024219,null,연예
5a429ac9588c1337b8eb59c8,"""신부님 앞에서 짧은 치마 입고 야한 춤 췄다""","{'대가대 병원', '행사', '장기 자랑', '간호사'}",1,0,102,1,0,2,2,0,null,http://v.media.daum.net/v/20171226135059528,null,사회
5a64385d588c134738c38055,은행원 10만명 벽 깨지나..연초부터 칼바람 부는 은행권,"{'은행권', '희망 퇴직', '부산', '금융', '직원', '모바일'}",2,102,1,0,1,2,0,0,null,http://v.media.daum.net/v/20180107132642425,null,경제
5a737323588c1303002fcef8,가상화폐 투자 실패 우울증 호소 20대 숨진 채 발견,"{'제작', '경찰', '발견', '부산', '가상화폐 투자'}",1,30,77,0,0,0,0,0,null,http://v.media.daum.net/v/20180201073756068,null,사회


## 실제 카테고리 : breakingnews
> 1개

In [36]:
lstOutDf2[lstOutDf2.real_category == 'breakingnews']

,title,extracted_keywords,정치,경제,사회,생활/문화,세계,IT/과학,연예,스포츠,real_category,link,real_category_from_naver,Decision
id,,,,,,,,,,,,,,
5a47dab9588c134d4010c937,"컬링에 쓰는 돌, 10년에 한 번만 캘 수 있다는데..","{'블루혼', '빙판', '화강암', '세계', '하우스', '경기', '컬링 스톤'}",0,3,0,14,49,38,1,3,breakingnews,http://v.media.daum.net/v/20171230030413747,스포츠,세계


## 실제 카테고리 : stock
> 8개

In [37]:
lstOutDf2[lstOutDf2.real_category == 'stock']

,title,extracted_keywords,정치,경제,사회,생활/문화,세계,IT/과학,연예,스포츠,real_category,link,real_category_from_naver,Decision
id,,,,,,,,,,,,,,
5a2b0aa2588c136a080754bd,비트코인 1200억원 어치를 쓰레기장에 버린 英남자,"{'가치', '허가', '비트코인 가격', '달러', '하드 디스크', '하웰즈'}",0,99,0,0,4,5,0,0,stock,http://v.media.daum.net/v/20171203112443874,경제,경제
5a2b0aa2588c136a080754ce,은행 예·적금 금리 줄줄이 오른다,"{'적금 금리', '금리 최고', '인상', '상품', '은행', '25'}",0,104,1,0,0,1,0,2,stock,http://v.media.daum.net/v/20171203203734684,경제,경제
5a3eb66e588c13083cf9bd3d,"비트코인, 1만400달러까지 폭락.. 얼마나 더 내려가나","{'CBOE', '거래', '변동성', '1만', 'CME', '달러', '비트코인...",0,106,0,0,1,1,0,0,stock,http://v.media.daum.net/v/20171223152348954,경제,경제
5a49385b588c132efc54ef93,"""그때 비트코인 샀어야지!""..세상에서 가장 우울한 사이트","{'구매', '비트코인', '가상 화폐', '만원', '아이맥', '문장', '사이트'}",0,61,0,0,1,46,0,0,stock,http://v.media.daum.net/v/20171231060023858,경제,경제
5a6452a7588c134a783a9b3a,"""연봉 맞춰줄게요""..가상화폐거래소 손짓에 어수선한 여의도","{'가상화폐 거래소', '빗썸', '가상 화폐 거래소', '러브콜'}",1,99,0,0,0,6,1,1,stock,http://v.media.daum.net/v/20180108060515842,경제,경제
5a65093e588c135a9c8cd7f1,"한국거래소, 오후 1시57분께 코스닥 '사이드 카' 발동","{'코스닥', '사이드카', '변동', '선물 가격'}",0,99,0,2,1,0,6,0,stock,http://v.media.daum.net/v/20180112140611341,경제,경제
5a650f6f588c131db07bf872,셀트리온 3형제 급락 '쇼크'..코스닥 890선 붕괴,"{'종목', '급락', '셀트리온 3형제', '상승', '각각', '거래일 대비'}",0,108,0,0,0,0,0,0,stock,http://v.media.daum.net/v/20180117155935641,경제,경제
5a737323588c1303002fcf03,"삼성전자 액면분할, 코스피 들었다 놨다","{'주식', '삼성전자 주가', '개인', '액면 분할'}",1,105,0,0,0,2,0,0,stock,http://v.media.daum.net/v/20180201030331563,경제,경제


## 실제 카테고리 : nation
> 182개

In [38]:
lstOutDf2[lstOutDf2.real_category == 'nation']

,title,extracted_keywords,정치,경제,사회,생활/문화,세계,IT/과학,연예,스포츠,real_category,link,real_category_from_naver,Decision
id,,,,,,,,,,,,,,
5a2a61bf588c13481c229d29,'탈의실 몰카' 혐의 전현직 국가대표 수영선수 5명 전원 무죄,"{'혐의', '이유', '기소', '진술', '범죄', '카메라', '판사', '무...",7,0,95,0,0,0,0,6,nation,http://v.media.daum.net/v/20171207111505668,사회,사회
5a2a61bf588c13481c229d2f,"""해경이 와야지""..해경, 생존자와 구조당국 통화 녹취록 공개","{'구조대', '해경이', '에어포켓', '녹취록', '상황', '사고', '통화'...",1,0,104,0,2,0,0,1,nation,http://v.media.daum.net/v/20171207112416100,사회,사회
5a2a61bf588c13481c229d34,3시간 빈소·40만원 장례비로 '가난한 죽음' 막을 수 있을까,"{'기초생활수급자', '서울시의회', '가족', '장례', '지원', '만원', '...",0,7,97,2,0,0,2,0,nation,http://v.media.daum.net/v/20171207172606594,사회,사회
5a2a61bf588c13481c229d37,"자유한국당, 안산시 의회서 '세월호 조례' 부결","{'시민', '한국당', '사회', '참사', '조례안', '문화', '부결', '...",68,1,36,1,1,1,0,0,nation,http://v.media.daum.net/v/20171207152607250,사회,정치
5a2aec8b588c132078bd207f,목줄 풀린 개가 산책하던 여성 습격..전치 3주 부상,"{'산책', '경찰', '부상', '전치', '종아리', '치료', '여성 습격',...",0,1,98,2,4,2,1,0,nation,http://v.media.daum.net/v/20171202154915532,사회,사회
5a2aec8b588c132078bd20a9,승객 136명 탄 여객선과 어선 충돌..어선 침몰 2명 부상,"{'신고', '선장', '어선', '여객선', '충돌', '해경', '여수시', '...",1,0,98,1,5,1,1,1,nation,http://v.media.daum.net/v/20171202110243754,사회,사회
5a2af415588c1376f0d67b02,등교길 점거한 '양심불량',"{'학생', '차량', '횡단보도 인도', '인천 부평구', '인정'}",1,2,78,15,4,1,7,0,nation,http://v.media.daum.net/v/20171206091857934,사회,사회
5a2af415588c1376f0d67b05,"""매일밤 음란 사이트 뒤져요"" 디지털 성범죄 끝없는 고통","{'디지털 성범죄', '피해자들', '고통', '성폭력', '유포'}",0,1,65,11,15,11,4,1,nation,http://v.media.daum.net/v/20171206060308272,사회,사회
5a2af415588c1376f0d67b17,"""민호야, 잊지않을게""..너무 이른 19살의 영결식","{'친구', '학교', '사고', '이군', '민호', '영결식', '현장 실습'}",3,0,104,0,1,0,0,0,nation,http://v.media.daum.net/v/20171206111141657,사회,사회


## 다음 뉴스의 카테고리중 네이버 뉴스의 카테고리와 겹치는 것에 대해서만 분석을 진행한다. 
* foreign : 세계
* culture : 생활/문화
* society : 사회
* economic : 경제
* digital : IT/과학
* politics : 정치
* 연예 : 연예
* 스포츠 : 스포츠
* stock : 경제

* nation, breakingnews, nan은 제외함. 
> * nation : ?
> * breakingnews :
>> 1건  
>> * 내용은 컬링 경기에서 사용되는 돌에 대한 기사 스포츠? 세계? -> 스포츠로 둔다. 
>* null
>> 언론사의 요청으로 기사가 삭제된 기사 3건  
>> 제휴종료된 언론사의 기사 1건

In [46]:
extCate = ~rcdf.real_category.isin(['null','nation', 'breakingnews'])
reRcdf = rcdf[extCate]

In [62]:
reOutdict = dict()
for idx, outcomeFile in enumerate(sorted(outcomeFileList)):
    fileName = os.path.split(outcomeFile)[1]
    df = pd.read_csv(outcomeFile, index_col = None, encoding='utf-8')
    df.set_index(keys='id', inplace=True)
    extDf = df[extCate]
    reOut = extDf.apply(lambda x: CounterOutcome(x, le), axis = 1)
    reOut = pd.DataFrame.from_records(reOut, columns=['id','정치','경제','사회','생활/문화','세계','IT/과학','연예','스포츠'])
    reOut.set_index(keys='id', inplace = True)
    modelIs, tagger = bm.ExtractModelType(outcomeFile)
    matchCount = (reOut.idxmax(axis = 1) == reRcdf.real_category_from_naver).sum()
    acc = accuracy_score(reRcdf.real_category_from_naver, reOut.idxmax(axis = 1))
    print (
    '''* {}. \n* Model : {}\n* tagger : {}\n* Match count : {} / {}\n* Accuracy score : {:.3f}'''.format(idx, modelIs, tagger, matchCount, reOut.shape[0], acc))
    byCls = extDf.loc[:,['LogisticRegression',
       'NeuralNetwork_1', 'NeuralNetwork_2', 'RandomForestClassifier',
       'SVC', 'XGBoost']].apply(lambda x: accuracy_score(reRcdf.real_category_from_naver, x))
    byCls = byCls.apply(lambda x: '{:.3f}'.format(x))
    print (byCls)
    print ()
    mx = '{}_{}'.format(modelIs, tagger)
    reOutdict[mx] = reOut

* 0. 
* Model : Doc2Vec_dbow
* tagger : twitter
* Match count : 7876 / 9185
* Accuracy score : 0.857
LogisticRegression        0.885
NeuralNetwork_1           0.803
NeuralNetwork_2           0.784
RandomForestClassifier    0.803
SVC                       0.888
XGBoost                   0.767
dtype: object

* 1. 
* Model : Doc2Vec_dbow
* tagger : mecab
* Match count : 7869 / 9185
* Accuracy score : 0.857
LogisticRegression        0.887
NeuralNetwork_1           0.789
NeuralNetwork_2           0.787
RandomForestClassifier    0.811
SVC                       0.892
XGBoost                   0.772
dtype: object

* 2. 
* Model : Doc2Vec_dm-c
* tagger : twitter
* Match count : 6190 / 9185
* Accuracy score : 0.674
LogisticRegression        0.707
NeuralNetwork_1           0.632
NeuralNetwork_2           0.560
RandomForestClassifier    0.640
SVC                       0.668
XGBoost                   0.584
dtype: object

* 3. 
* Model : Doc2Vec_dm-c
* tagger : mecab
* Match count : 6254 / 9185
* Ac

###  각 뉴스에 대하여 총 108개의 classifier가 분류한 Category 중에서 가장 많이 분류된 category가 실제 category와 일치하는 비율

In [63]:
reOutdf = pd.DataFrame()
for idx,key in enumerate(reOutdict):
    data = reOutdict[key]
    if reOutdf.shape == (0, 0):
        reOutdf = data.copy()
    else:
        reOutdf = reOutdf.add(data)

allAcc = accuracy_score(reRcdf.real_category_from_naver, reOutdf.idxmax(axis = 1))

matchCount = (reOutdf.idxmax(axis = 1) == reRcdf.real_category_from_naver).sum()
print ('Accuracy score : {:.4f}'.format(allAcc))
print ('Match count : {} / {}'.format(matchCount, reOutdf.shape[0]))
print (classification_report(reRcdf.real_category_from_naver, reOutdf.idxmax(axis = 1)))
outdf.head()

Accuracy score : 0.8573
Match count : 7874 / 9185
             precision    recall  f1-score   support

      IT/과학       0.27      0.75      0.40        65
         경제       0.60      0.71      0.65       331
         사회       0.79      0.72      0.75      1274
      생활/문화       0.40      0.77      0.53       218
         세계       0.52      0.84      0.65       330
        스포츠       1.00      0.92      0.95      3140
         연예       0.99      0.88      0.93      3079
         정치       0.73      0.86      0.79       748

avg / total       0.89      0.86      0.87      9185



,정치,경제,사회,생활/문화,세계,IT/과학,연예,스포츠
id,,,,,,,,
5a2a61bf588c13481c229d1e,0,1,25,21,60,1,0,0
5a2a61bf588c13481c229d1f,0,0,0,108,0,0,0,0
5a2a61bf588c13481c229d20,1,0,106,0,1,0,0,0
5a2a61bf588c13481c229d21,1,89,14,0,0,1,3,0
5a2a61bf588c13481c229d22,0,0,108,0,0,0,0,0


In [67]:
lstOutDf3 = extDf.loc[:,['title','extracted_keywords']]
lstOutDf3 = lstOutDf3.merge(reOutdf, right_index=True, left_index=True)
lstOutDf3.head()

,title,extracted_keywords,정치,경제,사회,생활/문화,세계,IT/과학,연예,스포츠
id,,,,,,,,,,
5a2a61bf588c13481c229d1e,"""밤이 무섭다""..비아그라 공장 연기에 남성들 부작용 호소","{'공장', '건강', '남성들', '지역', '부작용', '연기', '세보 효과'}",0,1,25,21,60,1,0,0
5a2a61bf588c13481c229d1f,식후 커피·늦은 양치질..점심식사 후 하면 안 좋은 습관 3가지,"{'치아', '건강', '입냄새', '커피', '점심 식사', '낮잠', '디스크'...",0,0,0,108,0,0,0,0
5a2a61bf588c13481c229d20,"'십년지기 생매장' 진짜 이유는..""'청부 통정' 알려질까 봐""","{'경찰', '성관계', '지인', '앙심', '진술', '철원', '아들', '주...",1,0,106,0,1,0,0,0
5a2a61bf588c13481c229d21,"신영자, 억 소리나는 갑질","{'신영자 이사장', '혐의', '롯데', '검찰', '유통업체', '징역', '매...",1,89,14,0,0,1,3,0
5a2a61bf588c13481c229d22,"""배신하지마"" 20대女 살인 피의자 유치장서 공범 남친에 쪽지","{'경찰', '혐의', '폭행', '쪽지', '유치장', '남자친구', '과자', ...",0,0,108,0,0,0,0,0


### 여러 classifier로 다음 뉴스를 네이버 category로 분류해보았을때, 모든 classifier가 하나의 cateogy로 분류한 비율

In [68]:
oneCategoryReOutput = lstOutDf3[reOutdf.eq(clsCount).sum(axis = 1) == 1]
oneCategoryReOutput.head()

,title,extracted_keywords,정치,경제,사회,생활/문화,세계,IT/과학,연예,스포츠
id,,,,,,,,,,
5a2a61bf588c13481c229d1f,식후 커피·늦은 양치질..점심식사 후 하면 안 좋은 습관 3가지,"{'치아', '건강', '입냄새', '커피', '점심 식사', '낮잠', '디스크'...",0,0,0,108,0,0,0,0
5a2a61bf588c13481c229d22,"""배신하지마"" 20대女 살인 피의자 유치장서 공범 남친에 쪽지","{'경찰', '혐의', '폭행', '쪽지', '유치장', '남자친구', '과자', ...",0,0,108,0,0,0,0,0
5a2a61bf588c13481c229d40,"""사장님과 식사할 땐 턱받이까지 매드려야 해요""","{'장기자랑', '직원들', '사례', '119', '제보', '병원', '직장 갑질'}",0,0,108,0,0,0,0,0
5a2a61bf588c13481c229d58,"'마이웨이' 박해미 ""아픔까지 안아 준 9세 연하 남편..결혼식 못 올렸다""","{'마이웨이', '인생', '박해미', '배우', '열정', '남편 황민', '사랑'}",0,0,0,0,0,0,108,0
5a2a61bf588c13481c229d77,"'감빵생활' 박해수, 정수정 버렸다..이규형x정해인 반전 [종합]","{'준호', '박해수', '감빵 생활', '제혁', '지호', '야구', '슬기'}",0,0,0,0,0,0,108,0


In [69]:
oneProp = oneCategoryReOutput.shape[0]/lstOutDf3.shape[0]
print ('Match count : {} / {}'.format(oneCategoryReOutput.shape[0], lstOutDf3.shape[0]))
print ('비율 : {0:.4f}, 약 {1:.1f}%'.format(oneProp, oneProp * 100))
print ()
oneAcc = accuracy_score(reRcdf.loc[oneCategoryReOutput.index.values, 'real_category_from_naver'], reOutdf.loc[oneCategoryReOutput.index.values].idxmax(axis = 1))
print ('Accuracy score : {:.4f}'.format(oneAcc))
print (classification_report(reRcdf.loc[oneCategoryReOutput.index.values, 'real_category_from_naver'], reOutdf.loc[oneCategoryReOutput.index.values].idxmax(axis = 1)))

Match count : 1024 / 9185
비율 : 0.1115, 약 11.1%

Accuracy score : 0.9854
             precision    recall  f1-score   support

      IT/과학       0.76      0.93      0.84        14
         경제       0.93      0.90      0.92        30
         사회       1.00      0.91      0.96        82
      생활/문화       0.75      1.00      0.86         6
         세계       1.00      0.93      0.97        15
        스포츠       1.00      1.00      1.00       663
         연예       0.99      0.99      0.99       154
         정치       0.91      0.98      0.94        60

avg / total       0.99      0.99      0.99      1024



### 여러 classifier로 다음 뉴스를 네이버 category로 분류해보았을때, 특정 category가 75%이상 나온 비율

In [70]:
x = reOutdf.divide(clsCount).multiply(100) >= 75
gt75 = lstOutDf3[x.sum(axis = 1) == 1]
gt75.head()

,title,extracted_keywords,정치,경제,사회,생활/문화,세계,IT/과학,연예,스포츠
id,,,,,,,,,,
5a2a61bf588c13481c229d1f,식후 커피·늦은 양치질..점심식사 후 하면 안 좋은 습관 3가지,"{'치아', '건강', '입냄새', '커피', '점심 식사', '낮잠', '디스크'...",0,0,0,108,0,0,0,0
5a2a61bf588c13481c229d20,"'십년지기 생매장' 진짜 이유는..""'청부 통정' 알려질까 봐""","{'경찰', '성관계', '지인', '앙심', '진술', '철원', '아들', '주...",1,0,106,0,1,0,0,0
5a2a61bf588c13481c229d21,"신영자, 억 소리나는 갑질","{'신영자 이사장', '혐의', '롯데', '검찰', '유통업체', '징역', '매...",1,89,14,0,0,1,3,0
5a2a61bf588c13481c229d22,"""배신하지마"" 20대女 살인 피의자 유치장서 공범 남친에 쪽지","{'경찰', '혐의', '폭행', '쪽지', '유치장', '남자친구', '과자', ...",0,0,108,0,0,0,0,0
5a2a61bf588c13481c229d23,"화성서 발견된 포탄 모양 둥근 물체, 정체는?","{'공개', '모양 물체', '사진', 'NASA', '학자', '암석', '화성'}",0,0,1,2,3,102,0,0


In [71]:
prop2 = gt75.shape[0]/lstOutDf3.shape[0]
print ('Match count : {} / {}'.format(gt75.shape[0], lstOutDf3.shape[0]))
print ('비율 : {0:.4f}, 약 {1:.1f}%'.format(prop2, prop2 * 100))
print ()
accis = accuracy_score(reRcdf.loc[gt75.index.values, 'real_category_from_naver'], reOutdf.loc[gt75.index.values].idxmax(axis = 1))
print ('Accuracy score : {:.4f}'.format(accis))
print (classification_report(reRcdf.loc[gt75.index.values, 'real_category_from_naver'], reOutdf.loc[gt75.index.values].idxmax(axis = 1)))

Match count : 7272 / 9185
비율 : 0.7917, 약 79.2%

Accuracy score : 0.9319
             precision    recall  f1-score   support

      IT/과학       0.49      0.82      0.61        56
         경제       0.75      0.77      0.76       266
         사회       0.91      0.77      0.83       951
      생활/문화       0.67      0.91      0.77       151
         세계       0.76      0.91      0.83       244
        스포츠       1.00      0.98      0.99      2649
         연예       0.99      0.98      0.99      2338
         정치       0.79      0.91      0.85       617

avg / total       0.94      0.93      0.93      7272



### 여러 classifier로 다음 뉴스를 네이버 category로 분류해보았을때, 특정 category가 50%보다 높게 나온 비율

In [72]:
x = reOutdf.divide(clsCount).multiply(100) > 50
gt50 = lstOutDf3[x.sum(axis = 1) == 1]
gt50.head()

,title,extracted_keywords,정치,경제,사회,생활/문화,세계,IT/과학,연예,스포츠
id,,,,,,,,,,
5a2a61bf588c13481c229d1e,"""밤이 무섭다""..비아그라 공장 연기에 남성들 부작용 호소","{'공장', '건강', '남성들', '지역', '부작용', '연기', '세보 효과'}",0,1,25,21,60,1,0,0
5a2a61bf588c13481c229d1f,식후 커피·늦은 양치질..점심식사 후 하면 안 좋은 습관 3가지,"{'치아', '건강', '입냄새', '커피', '점심 식사', '낮잠', '디스크'...",0,0,0,108,0,0,0,0
5a2a61bf588c13481c229d20,"'십년지기 생매장' 진짜 이유는..""'청부 통정' 알려질까 봐""","{'경찰', '성관계', '지인', '앙심', '진술', '철원', '아들', '주...",1,0,106,0,1,0,0,0
5a2a61bf588c13481c229d21,"신영자, 억 소리나는 갑질","{'신영자 이사장', '혐의', '롯데', '검찰', '유통업체', '징역', '매...",1,89,14,0,0,1,3,0
5a2a61bf588c13481c229d22,"""배신하지마"" 20대女 살인 피의자 유치장서 공범 남친에 쪽지","{'경찰', '혐의', '폭행', '쪽지', '유치장', '남자친구', '과자', ...",0,0,108,0,0,0,0,0


In [73]:
prop3 = gt50.shape[0]/lstOutDf3.shape[0]
print ('Match count : {} / {}'.format(gt50.shape[0], lstOutDf3.shape[0]))
print ('비율 : {0:.4f}, 약 {1:.1f}%'.format(prop3, prop3 * 100))
print ()
accis = accuracy_score(reRcdf.loc[gt50.index.values, 'real_category_from_naver'], reOutdf.loc[gt50.index.values].idxmax(axis = 1))
print ('Accuracy score : {:.4f}'.format(accis))
print (classification_report(reRcdf.loc[gt50.index.values, 'real_category_from_naver'], reOutdf.loc[gt50.index.values].idxmax(axis = 1)))

Match count : 8599 / 9185
비율 : 0.9362, 약 93.6%

Accuracy score : 0.8849
             precision    recall  f1-score   support

      IT/과학       0.36      0.77      0.49        64
         경제       0.65      0.72      0.68       320
         사회       0.83      0.73      0.78      1220
      생활/문화       0.48      0.81      0.61       198
         세계       0.59      0.87      0.70       313
        스포츠       1.00      0.94      0.97      2954
         연예       0.99      0.92      0.96      2811
         정치       0.74      0.88      0.81       719

avg / total       0.91      0.88      0.89      8599



### 여러 classifier로 다음 뉴스를 네이버 category로 분류해보았을때, 특정 category가 50%인 뉴스의 비율

In [74]:
x = reOutdf.divide(clsCount).multiply(100) == 50

eq50 = lstOutDf3[x.sum(axis = 1) == 1]
eq50.head()

,title,extracted_keywords,정치,경제,사회,생활/문화,세계,IT/과학,연예,스포츠
id,,,,,,,,,,
5a2aec8b588c132078bd207d,[뉴스+] 주말에 '집돌이·집순이' 되는 직장인들,"{'주말', '번아웃 증후군', '직장인', '스트레스', '증상', '교수', '...",0,2,54,48,3,1,0,0
5a2af415588c1376f0d67b16,'박근혜 5촌살인 의혹보도' 주진우·김어준 내일 대법원 선고,"{'사건', '연루', '혐의', '선고', '기자', '보도', '대통령', '조...",54,1,45,0,3,2,3,0
5a2b0aa2588c136a080754ec,"강남소방서 측 ""태연 가슴통증 호소..응급처치 특혜NO""","{'교통사고', '강남 소방서', '가슴 통증', '태연', '특혜'}",5,3,40,3,0,3,54,0
5a2b0eac588c13738825a5c2,"황우슬혜 ""꿀피부 비결은 아기 화장품, 비싼 것 필요 없어""","{'질문', '도전', '아기 화장품', '황우슬혜', '피부 관리', '역할'}",0,5,5,54,8,9,27,0
5a2c3b55588c135f3cb21ca8,"'이방인' 추신수 아내 하원미 ""삼남매 한방서 자는 것, 첫째 분리불안 때문""","{'김예은', '사진 이방인 캡처', '하원미', '경기', '삼남매', '이방인 ...",0,0,7,4,54,0,42,1


In [75]:
prop3 = eq50.shape[0]/lstOutDf3.shape[0]
print ('Match count : {} / {}'.format(eq50.shape[0], lstOutDf3.shape[0]))
print ('비율 : {0:.4f}, 약 {1:.1f}%'.format(prop3, prop3 * 100))
print ()
accis = accuracy_score(reRcdf.loc[eq50.index.values, 'real_category_from_naver'], reOutdf.loc[eq50.index.values].idxmax(axis = 1))
print ('Accuracy score : {:.4f}'.format(accis))
print (classification_report(reRcdf.loc[eq50.index.values, 'real_category_from_naver'], reOutdf.loc[eq50.index.values].idxmax(axis = 1)))

Match count : 50 / 9185
비율 : 0.0054, 약 0.5%

Accuracy score : 0.5000
             precision    recall  f1-score   support

      IT/과학       0.00      0.00      0.00         0
         경제       0.00      0.00      0.00         1
         사회       0.33      0.67      0.44         6
      생활/문화       0.00      0.00      0.00         1
         세계       0.12      0.50      0.20         2
        스포츠       1.00      0.67      0.80         9
         연예       1.00      0.48      0.65        25
         정치       0.67      0.33      0.44         6

avg / total       0.81      0.50      0.58        50



/Users/hyunyoun/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 여러 classifier로 다음 뉴스를 네이버 category로 분류해보았을때, 특정 category가 50%이상인 것이 하나도 없는 뉴스의 비율


In [76]:
x = reOutdf.divide(clsCount).multiply(100) >= 50

lt50 = lstOutDf3[x.sum(axis = 1) == 0]
lt50.head()

,title,extracted_keywords,정치,경제,사회,생활/문화,세계,IT/과학,연예,스포츠
id,,,,,,,,,,
5a2a61bf588c13481c229d4f,"문 대통령, '에너지제로 주택 신혼부부 입주세대 방문'","{'주택', '신혼부부', '서울 노원구', '대통령', '오픈 하우스', '에너지...",46,36,0,8,3,2,13,0
5a2a61bf588c13481c229d50,"[인터뷰]MBC 신동진 아나운서 ""6년만의 방송, 울컥하면 안되는데..""","{'부당 전보', '파업', '방송', '아나운서', '부서', '사장', '다시'}",2,4,32,24,1,20,25,0
5a2a61bf588c13481c229d5a,"[단독] ""윤계상, 합의 제안했다"" 탈세 주장 시위자, '무고죄 맞고소' (전문)","{'신고', '침대 업체', '허위', '소송', '주장', '윤계상 탈세'}",1,20,12,3,1,19,52,0
5a2a61bf588c13481c229d62,"이창명 ""그날 사건, 술 아닌 하늘의 경고""(인터뷰②)","{'편집', '스타', '이창명', '녹화', '만원', '전진학', '출발 드림팀...",5,9,33,17,3,4,37,0
5a2a61bf588c13481c229d6f,"'라디오스타' 손동운 ""회사 지분 51%, 매출 1위는 윤두준""","{'하이라이트 손동운', '51 매출', '멤버', '회사'}",1,17,3,0,6,36,45,0


In [77]:
prop3 = lt50.shape[0]/lstOutDf3.shape[0]
print ('Match count : {} / {}'.format(lt50.shape[0], lstOutDf3.shape[0]))
print ('비율 : {0:.4f}, 약 {1:.1f}%'.format(prop3, prop3 * 100))
print ()
accis = accuracy_score(reRcdf.loc[lt50.index.values, 'real_category_from_naver'], reOutdf.loc[lt50.index.values].idxmax(axis = 1))
print ('Accuracy score : {:.4f}'.format(accis))
print (classification_report(reRcdf.loc[lt50.index.values, 'real_category_from_naver'], reOutdf.loc[lt50.index.values].idxmax(axis = 1)))

Match count : 536 / 9185
비율 : 0.0584, 약 5.8%

Accuracy score : 0.4478
             precision    recall  f1-score   support

      IT/과학       0.00      0.00      0.00         1
         경제       0.14      0.50      0.21        10
         사회       0.26      0.46      0.33        48
      생활/문화       0.09      0.37      0.14        19
         세계       0.08      0.33      0.13        15
        스포츠       0.99      0.45      0.61       177
         연예       0.93      0.46      0.61       243
         정치       0.32      0.48      0.39        23

avg / total       0.79      0.45      0.54       536



* 특정 카테고리가 75%이상 나온 결과를 통해 확인해보면, 전체중 비율도 79%에 accuracy도 0.93정도 나오는 것을 확인할 수 있다. 
* 스포츠, 연예 분야는 precision, recall이 다른 카테고리보다 높게 나타나는 것을 확인할 수 있다. 